# Práctica 0 - Tratamiento de datos, particionamiento y diseño preliminar de la aplicación

_Pareja 6_
* David Kaack Sánchez
* Carlos César Rodríguez García


## Implementación de clase _Datos_

En esta implementación se utilizó _pandas_ para el almacenamiento de los datos.
Por cada columna se decide si contiene valores nominales o numéricos. Si son nominales se crea un mapeo de orden lexicográfico el cual se usará para reemplazar los valores nominales de la columna. Si los valores son numéricos se pasa a la siguiente columna. Adicionalmente, `nominalAtributos` guarda un arreglo booleano cuyos valores representan si la columna _i_ contiene valores nominales.
```python
class Datos:
    def __init__(self, nombreFichero: str):
        self.datosCrudos = pd.read_csv(nombreFichero)
        self.datos = self.datosCrudos.copy()

        self.nominalAtributos = []
        self.diccionarios = {}

        for columna in self.datos.columns:
            if self._es_nominal(columna):
                self.nominalAtributos.append(True)
                self.diccionarios[columna] = self._generar_mapeo(columna)
                self.datos[columna] = self._reemplazar_valores(columna)
            elif self._es_numerico(columna):
                self.nominalAtributos.append(False)
                self.diccionarios[columna] = {}
            else:
                raise ValueError(
                    f"La columna '{columna}' contiene valores que no son nominales ni enteros/decimales."
                )
```

Con el propósito de tener un registro histórico de los datos procesados, se guarda una versión cruda del dataset de entrada en el atributo `datosCrudos`.

In [1]:
from Datos import Datos

datos = Datos("heart_reduced.csv")
datos.datosCrudos

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,Class
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


El atributo `datos` contiene el mismo dataset que `datosCrudos` cambiando sus valores nominales por su representación en número.
El proceso de generar un mapeo entre los valores categóricos y números se hace en el constructor de la clase `Datos`.

In [2]:
datos.datos

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,Class
0,40,1,1,140,289,0,0,172,0,0.0,1,0
1,49,0,2,160,180,0,0,156,0,1.0,0,1
2,37,1,1,130,283,0,1,98,0,0.0,1,0
3,48,0,0,138,214,0,0,108,1,1.5,0,1
4,54,1,2,150,195,0,0,122,0,0.0,1,0
5,39,1,2,120,339,0,0,170,0,0.0,1,0
6,45,0,1,130,237,0,0,170,0,0.0,1,0
7,54,1,1,110,208,0,0,142,0,0.0,1,0
8,37,1,0,140,207,0,0,130,1,1.5,0,1
9,48,0,1,120,284,0,0,120,0,0.0,1,0


El atributo `diccionarios` guarda los mapeos para cada una de las columnas. Como se muestra a continuación, los valores numéricos para cada categoría se asignan en un orden lexicográfico. Por ejemplo, para la columna `ChestPainType` el atributo `ASY` que alfabéticamente se ordena antes de `ATA`, tiene un valor menor.

In [3]:
datos.diccionarios

{'Age': {},
 'Sex': {'F': 0, 'M': 1},
 'ChestPainType': {'ASY': 0, 'ATA': 1, 'NAP': 2},
 'RestingBP': {},
 'Cholesterol': {},
 'FastingBS': {},
 'RestingECG': {'Normal': 0, 'ST': 1},
 'MaxHR': {},
 'ExerciseAngina': {'N': 0, 'Y': 1},
 'Oldpeak': {},
 'ST_Slope': {'Flat': 0, 'Up': 1},
 'Class': {'0': 0, '1': 1}}

In [4]:
datos.nominalAtributos

[False, True, True, False, False, False, True, False, True, False, True, True]

Por último, el método `extraeDatos` obtiene las filas que se le indican a través de un array de índices.

In [5]:
datos.extraeDatos([5, 2, 3, 6])

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,Class
5,39,1,2,120,339,0,0,170,0,0.0,1,0
2,37,1,1,130,283,0,1,98,0,0.0,1,0
3,48,0,0,138,214,0,0,108,1,1.5,0,1
6,45,0,1,130,237,0,0,170,0,0.0,1,0


## Implementación de Estrategias de Particionado

**Validación Simple:**
Se pasa el número de ejecuciones y el porcentaje de datos proporcionados a la partición de test. Se crean particiones de modo aleatorio para representar todos los datos ya que podrían estar ordenados.
```python
class ValidacionSimple(EstrategiaParticionado):
    def __init__(self, numeroEjecuciones: int, proporcionTest: int):
        super().__init__()
        self.numeroEjecuciones = numeroEjecuciones
        self.proporcionTest = proporcionTest

    # Crea particiones segun el metodo tradicional de division de los datos segun el porcentaje deseado y el numero de ejecuciones deseado
    # Devuelve una lista de particiones (clase Particion)
    def creaParticiones(self, datos: pd.DataFrame, seed: int = 42) -> List[Particion]:
        n_filas = datos.shape[0]
        indices = list(range(n_filas))

        random.seed(seed)

        for _ in range(self.numeroEjecuciones):
            random.shuffle(indices)

            # se calcula el numero de ejemplos que se usaran como conjunto de prueba
            proporcion = floor(self.proporcionTest / 100 * n_filas)

            indices_test = indices[:proporcion]
            indices_train = indices[proporcion:]

            particion = Particion(indicesTrain=indices_train, indicesTest=indices_test)

            self.particiones.append(particion)

        return self.particiones
```


In [8]:
from EstrategiaParticionado import ValidacionSimple

estrategia = ValidacionSimple(numeroEjecuciones=5, proporcionTest=30)
particiones = estrategia.creaParticiones(datos.datos)

for i in range(len(particiones)):
    print(f"(Partición {i}) {particiones[i]}")

(Partición 0) indices train: [14, 12, 5, 2, 9, 3, 4, 11, 0, 1, 10]. indices test: [8, 13, 7, 6]
(Partición 1) indices train: [14, 5, 12, 4, 1, 9, 11, 6, 13, 10, 8]. indices test: [7, 0, 3, 2]
(Partición 2) indices train: [11, 8, 0, 10, 5, 12, 3, 7, 13, 14, 9]. indices test: [4, 1, 2, 6]
(Partición 3) indices train: [3, 7, 6, 9, 10, 4, 11, 12, 14, 8, 1]. indices test: [13, 5, 0, 2]
(Partición 4) indices train: [11, 9, 6, 14, 3, 8, 13, 5, 12, 2, 4]. indices test: [10, 0, 7, 1]


**Validación Cruzada:**
En el proceso de validación cruzada, se especifica el número de particiones deseadas, denotado como "k", y se generan "folds" o divisiones de los datos de longitud n/k. Por ejemplo, si tenemos un conjunto de datos con 10 filas y deseamos crear 5 particiones, cada "fold" contendría 2 elementos. Sin embargo, si n no es completamente divisible por k, las longitudes se distribuirán de manera equitativa entre los primeros "folds". Por ejemplo, si necesitamos 4 particiones para un conjunto de datos de 10 filas, los primeros 2 "folds" contendrían 3 elementos cada uno, mientras que los últimos 2 tendrían solo 2 elementos cada uno.

Se hacen _k_ iteraciones y para cada una de ellas se toma uno de los "folds" como set de prueba, mientras el resto se utiliza para entrenamiento.

```python
class ValidacionCruzada(EstrategiaParticionado):
    def __init__(self, numeroParticiones: int):
        self.numeroParticiones = numeroParticiones

    # Crea particiones segun el metodo de validacion cruzada.
    # El conjunto de entrenamiento se crea con las nfolds-1 particiones y el de test con la particion restante
    # Esta funcion devuelve una lista de particiones (clase Particion)
    def creaParticiones(self, datos: pd.DataFrame, seed: int = None) -> List[Particion]:
        n_filas = datos.shape[0]
        indices = list(range(n_filas))

        if seed is not None:
            random.seed(seed)
            random.shuffle(indices)

        longitud_fold = n_filas // self.numeroParticiones
        resto = n_filas % self.numeroParticiones

        inicio = 0

        for i in range(self.numeroParticiones):
            fin = inicio + longitud_fold

            # para las longitudes que no son divisibles enteramente por el numero
            # de particiones los primeros folds tienen una longitud extra.
            # e.g. datos = [1,2,3,4,5]; particiones = 3; folds = [[1,2], [3,4], [5]]
            if i < resto:
                fin += 1

            fin = min(fin, n_filas)

            # se construyen los indices para las particiones.
            # Los indices para training son todos aquellos que no pertenecen
            # a los indices de testing
            indices_test = indices[inicio:fin]
            indices_train = [
                indices[j] for j in range(n_filas) if j not in indices_test
            ]

            particion = Particion(indicesTrain=indices_train, indicesTest=indices_test)

            self.particiones.append(particion)

            inicio = fin

        return self.particiones
```

In [9]:
from EstrategiaParticionado import ValidacionCruzada

estrategia = ValidacionCruzada(numeroParticiones=5)
particiones = estrategia.creaParticiones(datos.datos)

for i in range(len(particiones)):
    print(f"(Partición {i}) {particiones[i]}")

(Partición 0) indices train: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]. indices test: [0, 1, 2]
(Partición 1) indices train: [0, 1, 2, 6, 7, 8, 9, 10, 11, 12, 13, 14]. indices test: [3, 4, 5]
(Partición 2) indices train: [0, 1, 2, 3, 4, 5, 9, 10, 11, 12, 13, 14]. indices test: [6, 7, 8]
(Partición 3) indices train: [0, 1, 2, 3, 4, 5, 6, 7, 8, 12, 13, 14]. indices test: [9, 10, 11]
(Partición 4) indices train: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]. indices test: [12, 13, 14]
